<a href="https://colab.research.google.com/github/nimanik21/central_bank_rl/blob/main/01_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
import plotly.express as px
from google.colab import drive

# Mount Google Drive
try:
    drive.mount('/content/drive')
except:
    pass


Mounted at /content/drive


In [ ]:
import json
import requests
import pandas as pd
import os

def fetch_fred_data(series_id, api_key, start=None, end=None):
    """
    Fetches a single FRED series in JSON format and returns a DataFrame.
    The final DataFrame is indexed by 'date' and has one 'value' column.
    """
    url = f"https://api.stlouisfed.org/fred/series/observations"
    url += f"?series_id={series_id}&api_key={api_key}&file_type=json"
    if start and end:
        url += f"&observation_start={start}&observation_end={end}"

    data = requests.get(url).json()
    if "observations" not in data:
        print(f"Error fetching {series_id}: Unexpected response format")
        return None

    df = pd.DataFrame(data["observations"])
    # Convert 'date' to datetime, 'value' to numeric
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    # Drop the realtime columns if they exist (avoid merge collisions)
    for col in ["realtime_start", "realtime_end"]:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)

    # Use 'date' as the index
    df.set_index("date", inplace=True)
    return df

def download_and_merge_fred_data(
    api_key,
    start="2004-01-01",
    end="2025-01-01",
    out_file="data/raw/macro_data_monthly.csv"
):
    """
    Downloads multiple important FRED series for a basic New Keynesian / macro analysis:
      - CPI (CPIAUCSL, monthly) - for inflation
      - UNRATE (monthly) - labor market
      - FEDFUNDS (monthly) - policy rate
      - INDPRO (monthly) - industrial production
      - M2SL (monthly) - money supply
      - T10YIE (daily) - 10-year breakeven inflation (TIPS-based inflation expectations)
      - GDPC1 (quarterly) - Real GDP
      - GDPPOT (quarterly) - Potential GDP

    We then merge and resample all data to a MONTHLY frequency, forward-filling
    any missing values (especially for quarterly or daily series).
    This is often used for a monthly VAR or for an RL environment that updates monthly.

    The final merged DataFrame is saved to CSV at 'out_file'.
    """

    # Dictionary of {FRED series ID : column name we want}
    series_dict = {
        # Monthly
        "CPIAUCSL":  "CPI",         # Consumer Price Index
        "UNRATE":    "UNRATE",      # Unemployment Rate
        "FEDFUNDS":  "FEDFUNDS",    # Federal Funds Rate
        "INDPRO":    "INDPRO",      # Industrial Production Index
        "M2SL":      "M2",          # M2 Money Stock

        # Daily
        #"T10YIE":    "T10YIEM",  # 10-Year Breakeven Inflation Rate (daily in FRED)

        # Quarterly
        "GDPC1":     "REAL_GDP",    # Real GDP
        "GDPPOT":    "POT_GDP",     # Potential GDP
    }

    dfs = {}
    for sid, colname in series_dict.items():
        print(f"Fetching {sid}...")
        df = fetch_fred_data(sid, api_key, start, end)
        if df is not None and not df.empty:
            # Rename 'value' to something descriptive
            df.rename(columns={"value": colname}, inplace=True)

            # Show a quick preview
            print(f"{sid} shape: {df.shape}")
            print(df.head(3), "\n")

            dfs[colname] = df
        else:
            print(f"Failed to fetch or empty data for {sid}")

    if not dfs:
        print("No data fetched at all. Exiting.")
        return

    # Merge all fetched DataFrames into one "master"
    merged_df = None
    for colname, df in dfs.items():
        if merged_df is None:
            merged_df = df
        else:
            # Outer join so we keep all dates from all series
            merged_df = merged_df.join(df, how="outer")

    print("Merged DF shape before resampling:", merged_df.shape)
    print(merged_df.head(10), "\n")

    # 1) Resample everything to end-of-month frequency ("M")
    # 2) Forward fill so daily/quarterly data extends through the month/quarter
    merged_df = merged_df.resample("M").ffill()

    # Restrict to [start, end] again
    merged_df = merged_df.loc[start:end]

    print("Merged DF shape after monthly resample+ffill:", merged_df.shape)
    print(merged_df.head(10), "\n")

    # Save to CSV
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    merged_df.to_csv(out_file)
    print(f"Saved final monthly data to {out_file}")

if __name__ == "__main__":
    # Example usage
    with open("/content/drive/MyDrive/API/api_keys.json", "r") as f:
        fred_api_key = json.load(f)["FRED_API_KEY"]

    # We'll produce a monthly CSV that includes inflation expectations,
    # money supply, industrial production, plus the standard inflation/unemp/GDP/FFR
    download_and_merge_fred_data(
        api_key=fred_api_key,
        start="2004-01-01",
        end="2025-01-01",
        out_file="/content/drive/MyDrive/central_bank_rl/data/raw/macro_data_monthly.csv"
    )


Fetching CPIAUCSL...
CPIAUCSL shape: (253, 1)
              CPI
date             
2004-01-01  186.3
2004-02-01  186.7
2004-03-01  187.1 

Fetching UNRATE...
UNRATE shape: (253, 1)
            UNRATE
date              
2004-01-01     5.7
2004-02-01     5.6
2004-03-01     5.8 

Fetching FEDFUNDS...
FEDFUNDS shape: (253, 1)
            FEDFUNDS
date                
2004-01-01      1.00
2004-02-01      1.01
2004-03-01      1.00 

Fetching INDPRO...
INDPRO shape: (253, 1)
             INDPRO
date               
2004-01-01  92.3268
2004-02-01  92.8995
2004-03-01  92.5368 

Fetching M2SL...
M2SL shape: (253, 1)
                M2
date              
2004-01-01  6082.2
2004-02-01  6121.9
2004-03-01  6158.0 

Fetching GDPC1...
GDPC1 shape: (84, 1)
             REAL_GDP
date                 
2004-01-01  15248.680
2004-04-01  15366.850
2004-07-01  15512.619 

Fetching GDPPOT...
GDPPOT shape: (85, 1)
                POT_GDP
date                   
2004-01-01  15349.36860
2004-04-01  15445.75949
200

<ipython-input-30-26f5d2b4ca37>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  merged_df = merged_df.resample("M").ffill()


In [ ]:
import json
import requests
import pandas as pd
import os

def fetch_fred_data(series_id, api_key, start=None, end=None):
    """
    Fetches a single FRED series in JSON format and returns a DataFrame.
    The final DataFrame is indexed by 'date' and has one column named after the series.
    """
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"
    if start and end:
        url += f"&observation_start={start}&observation_end={end}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for failed requests
        data = response.json()

        if "observations" not in data:
            print(f"❌ Error fetching {series_id}: Unexpected response format.")
            return None

        df = pd.DataFrame(data["observations"])
        df["date"] = pd.to_datetime(df["date"])
        df["value"] = pd.to_numeric(df["value"], errors="coerce")

        # Drop unnecessary columns
        df.drop(columns=["realtime_start", "realtime_end"], errors="ignore", inplace=True)

        # Set date as index
        df.set_index("date", inplace=True)

        return df.rename(columns={"value": series_id})  # Use series_id as column name

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed for {series_id}: {e}")
        return None

def download_and_merge_fred_data(api_key, start="2004-01-01", end="2025-01-01", out_file="data/raw/macro_data_monthly.csv"):
    """
    Fetches key FRED macroeconomic indicators, resamples them to a monthly frequency,
    and saves the final dataset as a CSV.
    """

    # Define FRED series to fetch
    series_dict = {
        "CPIAUCSL":  "CPI",         # Consumer Price Index (monthly)
        "UNRATE":    "UNRATE",      # Unemployment Rate (monthly)
        "FEDFUNDS":  "FEDFUNDS",    # Federal Funds Rate (monthly)
        "INDPRO":    "INDPRO",      # Industrial Production Index (monthly)
        "M2SL":      "M2",          # M2 Money Stock (monthly)
        "GDPC1":     "REAL_GDP",    # Real GDP (quarterly)
        "GDPPOT":    "POT_GDP",     # Potential GDP (quarterly)
    }

    all_data = {}  # Dictionary to store fetched dataframes

    # Fetch data for each series
    for sid, colname in series_dict.items():
        print(f"📡 Fetching {sid} ({colname})...")
        df = fetch_fred_data(sid, api_key, start, end)

        if df is not None and not df.empty:
            df.rename(columns={sid: colname}, inplace=True)
            all_data[colname] = df
            print(f"✅ {sid} fetched. Shape: {df.shape}")
        else:
            print(f"⚠️ Skipped {sid} due to missing data.")

    if not all_data:
        print("❌ No data was successfully fetched. Exiting.")
        return

    # Merge all fetched DataFrames
    merged_df = pd.concat(all_data.values(), axis=1, join="outer")

    print("\n📊 Merged Data Preview Before Resampling:")
    print(merged_df.head())

    # Resample to Monthly (Forward Fill for Quarterly Data)
    merged_df = merged_df.resample("M").ffill()

    # Ensure within the selected date range
    merged_df = merged_df.loc[start:end]

    print("\n✅ Final Merged Data (Monthly) - Shape:", merged_df.shape)
    print(merged_df.head())

    # Save to CSV
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    merged_df.to_csv(out_file)
    print(f"\n💾 Data saved to: {out_file}")

if __name__ == "__main__":
    # Load API key from a secure JSON file
    with open("/content/drive/MyDrive/API/api_keys.json", "r") as f:
        fred_api_key = json.load(f)["FRED_API_KEY"]

    # Fetch, merge, and save macroeconomic data
    download_and_merge_fred_data(
        api_key=fred_api_key,
        start="2004-01-01",
        end="2025-01-01",
        out_file="/content/drive/MyDrive/central_bank_rl/data/raw/macro_data_monthly.csv"
    )


📡 Fetching CPIAUCSL (CPI)...
✅ CPIAUCSL fetched. Shape: (253, 1)
📡 Fetching UNRATE (UNRATE)...
✅ UNRATE fetched. Shape: (253, 1)
📡 Fetching FEDFUNDS (FEDFUNDS)...
✅ FEDFUNDS fetched. Shape: (253, 1)
📡 Fetching INDPRO (INDPRO)...
✅ INDPRO fetched. Shape: (253, 1)
📡 Fetching M2SL (M2)...
✅ M2SL fetched. Shape: (253, 1)
📡 Fetching GDPC1 (REAL_GDP)...
✅ GDPC1 fetched. Shape: (84, 1)
📡 Fetching GDPPOT (POT_GDP)...
✅ GDPPOT fetched. Shape: (85, 1)

📊 Merged Data Preview Before Resampling:
              CPI  UNRATE  FEDFUNDS   INDPRO      M2  REAL_GDP      POT_GDP
date                                                                       
2004-01-01  186.3     5.7      1.00  92.3268  6082.2  15248.68  15349.36860
2004-02-01  186.7     5.6      1.01  92.8995  6121.9       NaN          NaN
2004-03-01  187.1     5.8      1.00  92.5368  6158.0       NaN          NaN
2004-04-01  187.4     5.6      1.00  92.8957  6199.1  15366.85  15445.75949
2004-05-01  188.2     5.6      1.00  93.5845  6275.9    

<ipython-input-3-974f63e359e7>:82: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  merged_df = merged_df.resample("M").ffill()



💾 Data saved to: /content/drive/MyDrive/central_bank_rl/data/raw/macro_data_monthly.csv
